In [1]:
import json
import logging
from typing import List, Dict, Any
from neopipe.pipeline import Pipeline
from neopipe.result import Result, Ok, Err

In [2]:

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
pipeline = Pipeline()

@pipeline.register(retries=3)
def process_fruit_data(fruit_data: List[Dict[str, Any]]) -> Result[Dict[str, Any], str]:
    try:
        unique_fruits = list(set(item['fruit'] for item in fruit_data))
        total_quantity = sum(item['quantity'] for item in fruit_data if item.get('quantity') is not None)
        return Ok({"unique_fruits": unique_fruits, "total_quantity": total_quantity})
    except (TypeError, KeyError) as e:
        return Err(f"Error processing fruit data: {str(e)}")

@pipeline.register(retries=3)
def calculate_average(data: Dict[str, Any]) -> Result[Dict[str, Any], str]:
    try:
        total_quantity = data["total_quantity"]
        unique_fruits = data["unique_fruits"]
        average_quantity = total_quantity / len(unique_fruits) if unique_fruits else 0
        return Ok({
            "unique_fruits": unique_fruits,
            "total_quantity": total_quantity,
            "average_quantity": average_quantity
        })
    except (TypeError, KeyError, ZeroDivisionError) as e:
        return Err(f"Error calculating average quantity: {str(e)}")

# Example event for local testing
if __name__ == "__main__":
    
    event = [
        {"fruit": "apple", "quantity": 10},
        {"fruit": "banana", "quantity": 20},
        {"fruit": "orange", "quantity": None}
    ]
    result = pipeline.run(event)
    print(result)

2024-06-27 01:12:42 - neopipe.pipeline - INFO - Pipeline started (UUID: 4070ce10-9e8d-41f8-9e08-ecc6d26024ae)
2024-06-27 01:12:42 - neopipe.task - INFO - Task process_fruit_data succeeded on attempt 1
2024-06-27 01:12:42 - neopipe.task - INFO - Task calculate_average succeeded on attempt 1


Result(value={'unique_fruits': ['banana', 'apple', 'orange'], 'total_quantity': 30, 'average_quantity': 10.0}, error=None)


In [3]:
pipeline.show_execution_plan()

Pipeline Execution Plan:
process_fruit_data : typing.List[typing.Dict[str, typing.Any]] -> neopipe.result.Result[typing.Dict[str, typing.Any], str]
 |
 V
calculate_average : typing.Dict[str, typing.Any] -> neopipe.result.Result[typing.Dict[str, typing.Any], str]


In [4]:
print(pipeline)

Pipeline with 2 tasks:
  Task(process_fruit_data, retries=3)
  Task(calculate_average, retries=3)
